Depending on Python's search paths, you may need to add additional paths and add the location of the license file that'll allow you to make full use of Gurobi. The "Python Libraries to Install" section of the chapter contains instructions on obtaining the license.

In [3]:
# import sys
# sys.path.append('$HOME/.local/lib/python3.10/site-packages')

# import os
# os.environ['GRB_LICENSE_FILE'] = "/util/opt/anaconda/deployed-conda-envs/packages/gurobi-python/gurobi.lic"

import gurobipy as gp
import numpy as np
import pickle
from src import *
from src.CreateEColiComm import *
from src.CreateCommModel import CreateCommModel
from src.parseXML import parseXML
from src.FindMaxGrowthRate import FindMaxGrowthRate
from src.RemoveBlocked import RemoveBlocked
from src.FluxCouplingAnalysis import FluxCouplingAnalysis

The first step is to load the original metabolic model, which is often stored in an XML file. This individual model will typically have multiple intracellular compartments and one extracellular compartments, with the former and latter representing the interior space of the organism and extracellular space respectively.

In [3]:
MetabolicMdl = parseXML('iAF1260.xml')
#Replacing the ±999999 with ±1000 lets this code work, probably because of getting rid of large coefficients, which may cause numerical instability
for Idx in range(len(MetabolicMdl[1])):
    if MetabolicMdl[1][Idx] < -1000:
        MetabolicMdl[1][Idx] = -1000
    if MetabolicMdl[2][Idx] > 1000:
        MetabolicMdl[2][Idx] = 1000

The output is a list of length 6. The elements are:
1. The stoichiometric matrix, a numerical matrix with dimensions M*N. Each row and column represents a metabolite and reaction respectively, so there are M rows and N columns. A negative element in the mth row and nth column means the nth reaction has the mth metabolite as a reactant. If it were positive, it's a product. If it were zero, that metabolite is uninvolved in the nth reaction.
2. The lower bounds, a numerical array of length N. The nth entries contain the lower bound to the nth reaction's flux.
3. The upper bounds, a numerical array of length N. The nth entries contain the upper bound to the nth reaction's flux.
4. The metabolites, a string array of length M. The mth entries contain the name for the mth metabolite.
5. The reactions, a string array of length N. The nth entries contain the name for the nth metabolite.
6. The reactions' expanded names. Like in field 5, the entries contain the names but a more descriptive version.

It's possible to modify the elements in the list. Care should be taken that a change in one field is reflected in the others. Below is an example of adding the reaction which we name "METt3pp", which transports L-Methionine from the cytoplasmic space to the periplasmic space.

In [4]:
MetabolicMdl[0] = np.hstack((MetabolicMdl[0],np.zeros( (MetabolicMdl[0].shape[0],1) ))) #Add a column to the stoichiometric matrix. Initially, this column is all zeros

MetabolicMdl[0][MetabolicMdl[3].index('met__L_c')][MetabolicMdl[0].shape[1] - 1] = MetabolicMdl[0][MetabolicMdl[3].index('h_c')][MetabolicMdl[0].shape[1] - 1] = -1 #Define the reactants
MetabolicMdl[0][MetabolicMdl[3].index('met__L_p')][MetabolicMdl[0].shape[1] - 1] = MetabolicMdl[0][MetabolicMdl[3].index('h_p')][MetabolicMdl[0].shape[1] - 1] = 1 #Define the products 

#Add newly added reactions' name
MetabolicMdl[4].append('METt3pp')
MetabolicMdl[5].append('METt3pp')

#Add new reactions' bounds
MetabolicMdl[1] = np.append(MetabolicMdl[1],0)
MetabolicMdl[2] = np.append(MetabolicMdl[2],1000)

In CreateEColiComm.py, there are functions that can use MetabolicMdl to create consortia where each member is auxotrophic in essential amino acids that the other members produce, thus making each member metabolically dependent on the other community members.

In [5]:
TheListOfMdls = TwoComm(MetabolicMdl)

With this we can create a community model. Because different naming conventions for exchange reactions exist, it's needed to provide the prefix each exchange reaction is associated with.

In [6]:
CommMdl = CreateCommModel(TheListOfMdls,"EX")

CommMdl is a list with the same format as MetabolicMdl.

In addition to preserving the intracellular and extracellular compartments of each community member, these members are all joined by a single community space, where metabolites may travel between it and each member's extracellular space. Thus, a metabolite excreted from one member travels from its extracellular space to the community space, then from there to another member's extracellular space where they can consume it. Similar to the individual model's exchange reactions, CommMdl contains community reactions that transport metabolites from and to the community space. Below, we modify their flux bounds to match the bounds found in the exchange reaction counterpart. Exchange reactions whose stoichiometries consist only of the community and extracellular metabolite respectively.

In [7]:
for Idx,Rxn in enumerate(MetabolicMdl[4]):
    if Rxn.startswith("EX"):
        TheIdx = CommMdl[4].index(Rxn + "_u")
        CommMdl[1][TheIdx] = MetabolicMdl[1][Idx]
        CommMdl[2][TheIdx] = MetabolicMdl[2][Idx]

The community model is ready for metabolic analysis. Below, we find the highest specific growth rate the community, and therefore what each of the member, can achieve. The parameter filled in by "BIOMASS" is a substring the function uses to identify biomass reactions.

In [8]:
GrowthRate,TheMdl,BioIndices = FindMaxGrowthRate(CommMdl,"BIOMASS")

Set parameter TokenServer to value "gurobi-license.pki.hcc.unl.edu"


925 3308
Final µ 0.7362334337085485 1.000000004027964 [0.5628394805561169, 0.43716052347184703]


GrowthRate contains the maximum predicted specific growth rate, TheMdl contains the linear model, which contains the decision variables of each reaction and the steady-state constraint, and BioIndices is a numerical array containing the index of each member's biomass reaction.

We can then proceed with Flux Coupling Analysis (FCA). The first step is to remove blocked reactions, reactions that are predict to be unable to achieve a nonzero flux. In other words, they can't be used. The growth rate to be used should be less than what FindMaxGrowthRate returned, due to numerical difficulties the LP solver may encounter when dealing with constraints that toe the edge of the feasible value.

In [9]:
TrimmedLP,TrimmedComm,FVT = RemoveBlocked(TheMdl,CommMdl,GrowthRate*.99,BioIndices)

Removing variables <gurobi.Var BIOMASS_Ec_iAF1260_core_59p81M_c0 (value 0.36868782724893495)> <gurobi.Var BIOMASS_Ec_iAF1260_core_59p81M_c1 (value 0.36018327212252815)> 3165 {0.0, 0.7288710993714631}


The results can be saved as seen below to avoid the need to rerun RemoveBlocked.

In [ ]:
TrimmedLP.write("TrimmedLP.mps")
with open("TrimmedData.pkl",'wb') as File:
    pickle.dump((TrimmedComm,FVT),File)

After that, the actual determination of reaction dependencies can begin.

In [ ]:
FCTbl = FluxCouplingAnalysis(TrimmedLP,FVT)
with open("FCAResults.pkl","wb") as File:
    pickle.dump(FCTbl,File)

FluxCouplingAnalysis
